In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(4)


In [3]:
#df['B_p_reject'] = np.random.randint(0,2,size=(len(df)))

In [4]:
df['B_p_reject']

0        1
1        1
2        1
3        1
4        1
        ..
99995    1
99996    1
99997    1
99998    0
99999    1
Name: B_p_reject, Length: 100000, dtype: int64

In [5]:
df['B_p_stratum'] = df['B_p'].astype(str) + df['B_p_reject'].astype(str)



In [6]:
df['B_p_stratum'].value_counts()

11    61667
00    30170
01     8056
10      107
Name: B_p_stratum, dtype: int64

In [7]:
# = 
#df['B_p_reject'] = df['B_p']

In [8]:
FRAC_ADMIT = df[['A']].sum()/len(df)


In [9]:
FRAC_ADMIT

A    0.39203
dtype: float64

In [10]:
len(df)

100000

In [11]:
#df['ml_outcomes'] = df['R']

In [12]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision,B_p
26877,0,20,0.0548,16,20,-3.291633,-2.846771,0,0
61685,0,23,0.0716,19,23,-3.007015,-2.562153,0,0
64436,0,24,0.0782,20,24,-2.912143,-2.467280,0,0
96092,0,24,0.0782,21,24,-2.817270,-2.467280,0,0
69517,0,24,0.0782,19,24,-3.007015,-2.467280,0,0
...,...,...,...,...,...,...,...,...,...
27554,1,102,1.2424,102,118,4.867418,6.450753,1,1
73760,1,104,1.2437,104,121,5.057164,6.735371,1,1
61344,1,107,1.2452,107,125,5.341782,7.114862,1,1
18644,1,107,1.2452,107,124,5.341782,7.019989,1,1


In [13]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [14]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','B_p_reject','ml_outcomes']].groupby(['R','T','B_p','B_p_reject']).sum().reset_index()

In [15]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p','B_p_reject']].groupby(['R','T','B_p','B_p_reject']).count().reset_index()
df_count.columns = ['R','T','B_p','B_p_reject','Count']
df_count['N'] = df_count['Count']

In [16]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [17]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N
0,499,0,0,20,0,0,0.0548,1
1,498,1,0,23,0,0,0.0716,1
4,497,4,0,25,0,1,0.0853,1
6,496,6,0,26,0,1,0.0930,1
7,495,7,0,26,1,1,0.0930,1
9,494,9,0,27,1,1,0.1013,1
13,493,13,0,29,0,1,0.1199,1
2,492,2,0,24,0,0,0.2346,3
41,491,41,0,38,1,0,0.2425,1
51,490,51,0,41,1,0,0.2985,1


### Setup optimization problem 

In [18]:
from ortools.linear_solver import pywraplp


In [19]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [20]:
len(dff[['R','T','B_p','B_p_reject']])

500

In [21]:
dff

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N
0,499,0,0,20,0,0,0.0548,1
1,498,1,0,23,0,0,0.0716,1
2,492,2,0,24,0,0,0.2346,3
3,484,3,0,25,0,0,0.3412,4
4,497,4,0,25,0,1,0.0853,1
5,478,5,0,26,0,0,0.4650,5
6,496,6,0,26,0,1,0.0930,1
7,495,7,0,26,1,1,0.0930,1
8,465,8,0,27,0,0,0.7091,7
9,494,9,0,27,1,1,0.1013,1


In [22]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'], row['B_p_reject'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [23]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [24]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

39203


In [25]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [26]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'], row['B_p_reject'])]
    key2s = [(row['R'],row['T'], 1-row['B_p'], 1-row['B_p_reject']), (row['R'],row['T'], row['B_p'], 1-row['B_p_reject']), (row['R'],row['T'], 1-row['B_p'], row['B_p_reject'])]
    
    for key2 in key2s:
        constrain_bp = solver.Constraint(0.0, 0.0)
        
        if key2 not in vars_cache:
            didntexist+=1
            continue
            
        var2 = vars_cache[key2]
        
        constrain_bp.SetCoefficient(var1, -1.0)
        constrain_bp.SetCoefficient(var2, 1.0)
        exists+=1

didntexist, exists

(424, 1076)

In [27]:
white_pass_boards_pass_boards_reject = []
white_fail_boards_pass_boards_reject = []
black_pass_boards_pass_boards_reject = []
black_fail_boards_pass_boards_reject = []
white_pass_boards_fail_boards_reject = []
white_fail_boards_fail_boards_reject = []
black_pass_boards_fail_boards_reject = []
black_fail_boards_fail_boards_reject = []


for key in vars_cache:
    r, t, b_p, b_p_reject = key
    if b_p == 1 and r==0 and b_p_reject==1:
        white_pass_boards_pass_boards_reject.append(key)
    elif b_p == 0 and r==0 and b_p_reject==1:
        white_fail_boards_pass_boards_reject.append(key)
    elif b_p == 1 and r==1 and b_p_reject==1:
        black_pass_boards_pass_boards_reject.append(key)
    elif b_p == 0 and r==1 and b_p_reject==1:
        black_fail_boards_pass_boards_reject.append(key)
        
    if b_p == 1 and r==0 and b_p_reject==0:
        white_pass_boards_fail_boards_reject.append(key)
    elif b_p == 0 and r==0 and b_p_reject==0:
        white_fail_boards_fail_boards_reject.append(key)
    elif b_p == 1 and r==1 and b_p_reject==0:
        black_pass_boards_fail_boards_reject.append(key)
    elif b_p == 0 and r==1 and b_p_reject==0:
        black_fail_boards_fail_boards_reject.append(key)

len(white_pass_boards_pass_boards_reject),len(white_fail_boards_pass_boards_reject),len(black_pass_boards_pass_boards_reject),len(black_fail_boards_pass_boards_reject)




(99, 66, 81, 59)

In [28]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p','B_p_reject']].groupby(['R','B_p','B_p_reject']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'],row['B_p_reject'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'],row['B_p_reject'])] = row['N']

In [29]:
# Now we have one constraint
solver.NumConstraints()

1501

In [30]:
#Of those who pass the boards exams, pass boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards_pass_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_pass_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_pass_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_pass_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [31]:
#Of those who fail the boards exams, pass boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards_pass_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_fail_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_pass_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards_pass_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_pass_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [32]:
constrain_pass_boards_fail_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_pass_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_fail_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_pass_boards_fail_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))

    
#Of those who fail the boards exams, fail boards exams if rejected
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards_fail_boards_reject = solver.Constraint(0.0, 0.0)

for key in white_fail_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_fail_boards_reject.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards_fail_boards_reject:
    r, t, b_p, b_p_reject = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p, b_p_reject)]
    N_TOTAL = NUM_TOTALS[(r,b_p, b_p_reject)]
    
    constrain_fail_boards_fail_boards_reject.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Add constraints on people who fail boards exams

## Solve linear program

In [33]:
solver.ABNORMAL

4

In [34]:
status = solver.Solve()


In [35]:
status

0

In [36]:
solver.OPTIMAL

0

In [37]:
applicant_stratum

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [38]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [39]:
df_decisions

,row_id,decision
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.000000
7,7,0.000000
8,8,0.000000
9,9,0.000000


In [40]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
143,0,143,0,64,1,1,1166.6304,1476,143,1.000000
147,1,147,0,65,1,1,1156.1795,1435,147,1.000000
135,2,135,0,62,1,1,1154.1252,1524,135,1.000000
131,3,131,0,61,1,1,1151.2458,1557,131,1.000000
151,4,151,0,66,1,1,1146.6396,1398,151,1.000000
139,5,139,0,63,1,1,1141.3182,1474,139,1.000000
155,6,155,0,67,1,1,1139.6679,1367,155,1.000000
123,7,123,0,59,1,1,1089.1189,1553,123,0.000000
127,8,127,0,60,1,1,1075.2844,1492,127,0.000000
159,9,159,0,68,1,1,1064.8970,1258,159,1.000000


In [41]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
0,499,0,0,20,0,0,0.0548,1,0,0.000000
274,489,274,1,20,0,0,0.3016,1,274,1.000000
275,417,275,1,21,0,0,1.2256,4,275,1.000000
277,488,277,1,22,0,1,0.3117,1,277,0.000000
276,487,276,1,22,0,0,0.3117,1,276,0.000000
1,498,1,0,23,0,0,0.0716,1,1,0.000000
279,472,279,1,23,1,1,0.6348,2,279,1.000000
278,372,278,1,23,0,0,2.5392,8,278,1.000000
2,492,2,0,24,0,0,0.2346,3,2,0.000000
280,449,280,1,24,0,0,0.9708,3,280,1.000000


In [42]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,B_p_reject,ml_outcomes,N,row_id,decision
143,0,143,0,64,1,1,1166.6304,1476,143,1.000000
147,1,147,0,65,1,1,1156.1795,1435,147,1.000000
135,2,135,0,62,1,1,1154.1252,1524,135,1.000000
131,3,131,0,61,1,1,1151.2458,1557,131,1.000000
151,4,151,0,66,1,1,1146.6396,1398,151,1.000000
139,5,139,0,63,1,1,1141.3182,1474,139,1.000000
155,6,155,0,67,1,1,1139.6679,1367,155,1.000000
123,7,123,0,59,1,1,1089.1189,1553,123,0.000000
127,8,127,0,60,1,1,1075.2844,1492,127,0.000000
159,9,159,0,68,1,1,1064.8970,1258,159,1.000000


In [43]:
xxx.to_csv('./decision.csv')

In [44]:
solution = pd.read_csv('./decision.csv')

In [45]:
len(xxx)

500

In [46]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)+'_'+df['B_p_reject'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)+'_'+xxx['B_p_reject'].astype(str)



In [47]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT['A']

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [48]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Principal Fairness',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()